# Gérer des tables avec l'extension [Pandas](https://pandas.pydata.org/docs/)

## Pandas apporte une nouvelle structure de données complétant les précédentes

Avant de commencer l'explication de qu'apporte [Pandas](https://pandas.pydata.org/docs/), voici un petit rappel de ce qui a été vu précédemment.

Le langage Python gère des données de type simple comme les entiers ou les flottants. Sur la base de ces types, Python construit des types composites, qui sont les listes (avec le cas particulier des tuples) et les dictionnaires. Les listes rassemblent plusieurs données séparées par des virgules dans une structure ordonnée (par le numéro d'index, qui est toujours sous-entendu). Les dictionnaires associent une série de valeurs avec des clés (c'est pourquoi on trouve aussi en anglais le terme *associative arrays* pour qualifier ce type de structure). Ces types composites peuvent ensuite s'enchâsser les uns dans les autres et on reconnaît les listes des dictionnaires car le premier est entouré de parenthèses, le second d'accolades.

L'extension [NumPy](01_numpy.ipynb) introduit un autre type de structure appelée `ndarray`, qui veut dire vecteur à `n` dimensions. Nous dirions plutôt des matrices en français. La caractéristique de cette structure est que tous ses éléments sont du même type et ce type est numérique, soit un type entier, soit un type flottant. La contre-partie est une plus grande efficacité dans ce qui ne serait autrement que des listes en-chassées les unes dans les autres.

L'extension [Pandas](https://pandas.pydata.org/docs/) introduit maintenant un autre type de structure aussi très courant qui sont les tables de données, appelées ici des `dataframes`. Il s'agit des tables couramment utilisées dans les bases de données relationnelles ou les feuilles de calcul d'un tableur. Les données sont rangées en colonne, chaque colonne ne contenant qu'un seul type de données. Les enregistrements se lisent ensuite ligne par ligne, chaque enregistrement étant référencée par un index (comme pour les éléments d'une liste), qui correspond à une première colonne particulière.

Parmi ces `dataframes`, un sous ensemble bénéficie d'un traitement particulier. Il s'agit des enregistrements dont l'index est une une [date/heure](https://docs.python.org/fr/3/library/datetime.html), format particulier contenant la date et l'heure exact allant jusqu'à la précision qu'on veut et qui au final peut se ramener à un type numérique, qui permet d'ordonner les enregistrements. Pandas offre des foncitonalités supplémentaires pour ces `dataframes` particuliers.


## Déclarer des dataframes

Les `dataframe` de Pandas se construisent donc par défaut colonne par colonne.

In [1]:
import pandas as pd

DF = pd.DataFrame({'A': [10, 20, 30], 
                   'B': [32, 42, 31],
                   'C': ["str", "dgr", "ftr"]
                   })
DF

,A,B,C
0,10,32,str
1,20,42,dgr
2,30,31,ftr


Mais quand certaines colonnes ne contiennent que des valeurs identiques, on peut simplifier l'entrée.

In [2]:
import numpy as np

df2 = pd.DataFrame(
    {
        "A": 1.0,
        "B": pd.Timestamp("20130102"),
        "C": pd.Series(1, index=list(range(4)), dtype="float32"),
        "D": np.array([3] * 4, dtype="int32"),
        "E": pd.Categorical(["test", "train", "test", "train"]),
        "F": "foo",
    }
)
df2

,A,B,C,D,E,F
0,1.0,2013-01-02,1.0,3,test,foo
1,1.0,2013-01-02,1.0,3,train,foo
2,1.0,2013-01-02,1.0,3,test,foo
3,1.0,2013-01-02,1.0,3,train,foo


Voici un autre exemple.

In [3]:
df = pd.DataFrame(
    {
        "Name": [
            "Braund, Mr. Owen Harris",
            "Allen, Mr. William Henry",
            "Bonnell, Miss. Elizabeth",
        ],
        "Age": [22, 35, 58],
        "Sex": ["male", "male", "female"],
    }
)
df

,Name,Age,Sex
0,"Braund, Mr. Owen Harris",22,male
1,"Allen, Mr. William Henry",35,male
2,"Bonnell, Miss. Elizabeth",58,female


## Lien entre les dataframe et les fichiers JSON

Cet exemple montre la parenté étroite avec le format [JSON](https://json.org/json-fr.html) qui nous avons vu à l'occasion de la [gestion des fichiers](../01_les_bases/02_fichiers.ipynb). Il existe dans Pandas une méthode pour écrire un dataframe dans un fichier JSON ([to_json](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_json.html)) ou ensuite le lire ([read_json](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_json.html)).

In [10]:
# écriture du dataframe précédant dans un fichier json
with open('persons.json', 'w') as file:
    df.to_json(file, indent = 4)

# relecture brute du fichier json et impression du tampon lu
with open('persons.json') as file:
    tampon = file.read()

print(tampon)

{
    "Name":{
        "0":"Braund, Mr. Owen Harris",
        "1":"Allen, Mr. William Henry",
        "2":"Bonnell, Miss. Elizabeth"
    },
    "Age":{
        "0":22,
        "1":35,
        "2":58
    },
    "Sex":{
        "0":"male",
        "1":"male",
        "2":"female"
    }
}


Ces deux fonctions contiennent plus d'option, qui correspondent à toutes les options possibles quand on écrit des types numériques dans un fichier texte.

Pour cette raison, il existe format binaire correspondant à ces structures, le format [bson](https://github.com/py-bson/bson) qui se charge avec l'extension du même nom. Ce type de format est abondamment utilisé par la base de données [mongoDB](https://www.mongodb.com/fr-fr).

## Manipuler les dataframes

On peut extraire des colonnes.

In [8]:
df["Age"]

0    22
1    35
2    58
Name: Age, dtype: int64

`df["Age"]` est une série Pandas. On remarquera que chaque élément est toujours indexé. Pour transformer cette série en véritable liste Python, il faut utiliser la méthode `tolist`.

In [9]:
df["Age"].tolist()

[22, 35, 58]

Et si on veut obtenir une suite d'éléments, il faut utiliser l'astérisque.

In [14]:
print(*df["Age"].tolist())

22 35 58


## Les dataframes pour des évènements temporels

Parmis les dataframes, il existe ceux qui dont la première colonne est un temps. C'est un cas courant, car il peut s'agir de relevés temporels.

In [6]:
dates = pd.date_range("20230101", periods=6)
dates

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06'],
              dtype='datetime64[ns]', freq='D')

In [15]:
import numpy as np

rng = np.random.default_rng(3)
df = pd.DataFrame(rng.random((6,4), dtype='float32'),
                  index= dates,
                  columns = list('ABCD'))
df


,A,B,C,D
2023-01-01,0.811504,0.085649,0.179441,0.236811
2023-01-02,0.181365,0.801274,0.869233,0.582162
2023-01-03,0.039399,0.094129,0.332202,0.433127
2023-01-04,0.621228,0.479051,0.264788,0.159739
2023-01-05,0.691417,0.734577,0.032688,0.113672
2023-01-06,0.452127,0.391228,0.887825,0.516740


Comme avec une table de base de données, on peut choisir de n'afficher que les premiers ou les derniers éléments (par défaut ces commandes montrent cinq éléments, s'il n'y pas de paramètres).

In [16]:
df.head(3)

,A,B,C,D
2023-01-01,0.811504,0.085649,0.179441,0.236811
2023-01-02,0.181365,0.801274,0.869233,0.582162
2023-01-03,0.039399,0.094129,0.332202,0.433127


In [17]:
df.tail()

,A,B,C,D
2023-01-02,0.181365,0.801274,0.869233,0.582162
2023-01-03,0.039399,0.094129,0.332202,0.433127
2023-01-04,0.621228,0.479051,0.264788,0.159739
2023-01-05,0.691417,0.734577,0.032688,0.113672
2023-01-06,0.452127,0.391228,0.887825,0.516740


On peut aussi retrouver les index.

In [18]:
df.index

DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05', '2023-01-06'],
              dtype='datetime64[ns]', freq='D')

ou les noms de colonnes

In [19]:
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

On peut convertir les dataframes pandas en ndarray numpy. Mais une différence fondamentale est que les éléments d'un ndarray sont tous de même type alors que les éléments d'un dataframe peuvent avoir plusieurs types, chaque colonne étant d'un type donné. La conversion est donc simple et immédiate si le dataframe a déjà la structure d'un ndarray, typiquement, s'il n'est constitué que de flottants.

In [20]:
df.to_numpy()

array([[0.8115045 , 0.08564913, 0.17944068, 0.2368105 ],
       [0.18136477, 0.8012744 , 0.8692326 , 0.582162  ],
       [0.03939909, 0.09412861, 0.33220154, 0.43312693],
       [0.6212278 , 0.4790513 , 0.2647882 , 0.1597389 ],
       [0.6914169 , 0.7345771 , 0.03268814, 0.11367202],
       [0.45212662, 0.39122814, 0.88782495, 0.51674014]], dtype=float32)

on remarquera que les index ont disparus ainsi que les noms de colonnes.